# 네이버 부동산

http://land.naver.com 데이터 크롤링

<img width="80%" src="http://i.imgur.com/PnXGPXm.png" >

2017 FinanceData http://fb.com/financedata    
<!-- TEASER_END -->

http://land.naver.com 의 검색 인자는 다음과 같이 구성

http://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=all&hscpTypeCd=A01:A03:A04&cortarNo=1168010600

* rletTypeCd: A01=아파트, A02=오피스텔, B01=분양권, 주택=C03, 토지=E03, 원룸=C01, 상가=D02, 사무실=D01, 공장=E02, 재개발=F01, 건물=D03
* tradeTypeCd (거래종류): all=전체, A1=매매, B1=전세, B2=월세, B3=단기임대
* hscpTypeCd (매물종류): 아파트=A01, 주상복합=A03, 재건축=A04 (복수 선택 가능)
* cortarNo(법정동코드): (예: 1168010600 서울시, 강남구, 대치동)

※ 법정동 코드에 대한 상세내용은 https://goo.gl/P6ni8Q 참조

# 사이 문자열 검색

정규식을 사용하여 특정 문자열 사이에 있는 문자열을 얻는다


In [1]:
import re

text = '220/191공급면적220.92㎡전용면적191.06㎡'

공급면적 = re.findall('공급면적(.*?)㎡', text)[0]
전용면적 = re.findall('전용면적(.*?)㎡', text)[0]

print('공급면적: ', 공급면적) 
print('전용면적: ', 전용면적) 

공급면적:  220.92
전용면적:  191.06


# 특정 페이지 테이블

지역을 검색하는 경우 다음과 같은 형태의 URL가 된다

http://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04&cortarNo=1168010600

이 페이지의 아래 쪽 테이블(table)을 분석하여 데이터를 추출해보자

<img src="http://i.imgur.com/GGP6xrP.png" >


In [2]:
import re
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import requests

url = 'http://land.naver.com/article/articleList.nhn?rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04&cortarNo=1168010600'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')

table = soup.find('table')
trs = table.tbody.find_all('tr')

# 거래, 종류, 확인일자, 확인주석, 매물명, 면적(㎡), 층, 매물가(만원), 연락처
for tr in trs[::2]:
    tds = tr.find_all('td')
    cols = [' '.join(td.text.strip().split()) for td in tds]
    
    if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
        cols.insert(3, '')

    거래 = cols[0]
    종류 = cols[1]
    확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
    현장확인 = cols[3]
    확인주석 = cols[4]
    면적 = cols[5]
    공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
    전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
    공급면적 = float(공급면적)
    전용면적 = float(전용면적)
    층 = cols[6]
    매물가 = int(cols[7].replace(',', '')) 
    연락처 = cols[8]
        
    print(거래, 종류, 확인일자, 현장확인, 확인주석, 공급면적, 전용면적, 층, 매물가, 연락처)


매매 아파트 2017-08-04 00:00:00 5/14 현장확인 대치SK뷰 110.89 84.38 4/14 170000 우리공인중개사02-565-0066
매매 아파트 2017-08-03 00:00:00 5/13 현장확인 선경1,2차 182.22 160.76 7/15 265000 아성공인중개사사무소02-555-9918
매매 아파트 2017-08-01 00:00:00 5/13 현장확인 개포우성1차 102.48 84.81 3/15 178000 황금공인중개사사무소02-558-4949
매매 아파트 2017-08-01 00:00:00 5/14 현장확인 선경1,2차 137.45 128.32 8/15 207000 황금공인중개사사무소02-558-4949
매매 아파트 2017-08-01 00:00:00 5/13 현장확인 한보미도맨션1,2차 220.92 191.06 8/14 320000 미도금성공인중개사사무소02-3453-8885
매매 아파트 2017-08-01 00:00:00 5/14 현장확인 대치SK뷰 110.89 84.38 16/17 170000 대치SK VIEW공인중개사사무소02-555-0230
매매 아파트 2017-07-28 00:00:00 5/14 현장확인 선경1,2차 148.6 127.75 5/15 220000 대성공인중개사사무소02-556-8888
매매 아파트 2017-07-28 00:00:00 5/14 현장확인 대치SK뷰 122.42 93.4 10/17 187000 금강공인중개사사무소02-565-8088
매매 아파트 2017-07-25 00:00:00 5/13 현장확인 대치SK뷰 122.42 93.4 6/20 186000 우리공인중개사02-565-0066
매매 아파트 2017-07-25 00:00:00 5/13 현장확인 대치SK뷰 145.94 112.36 5/14 240000 삼성플러스공인중개사사무소02-554-5985
매매 아파트 2017-07-24 00:00:00 5/11 현장확인 선경1,2차 157.36 146.75 6/15 240000 아성공인중개사사무소

# 함수로 만들기
앞에서 확인된 내용으로 재사용 가능한 함수를 만든다

In [3]:
import re
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup

def get_naver_realasset(area_code):
    url = 'http://land.naver.com/article/articleList.nhn?' \
        + 'rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04' \
        + '&cortarNo=' + area_code

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    table = soup.find('table')
    trs = table.tbody.find_all('tr')
    
    value_list = []

    # 거래, 종류, 확인일자, 확인주석, 매물명, 면적(㎡), 층, 매물가(만원), 연락처
    for tr in trs[::2]:
        tds = tr.find_all('td')
        cols = [' '.join(td.text.strip().split()) for td in tds]

        if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
            cols.insert(3, '')

        거래 = cols[0]
        종류 = cols[1]
        확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
        현장확인 = cols[3]
        확인주석 = cols[4]
        면적 = cols[5]
        공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
        전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
        공급면적 = float(공급면적)
        전용면적 = float(전용면적)
        층 = cols[6]
        매물가 = int(cols[7].replace(',', '')) 
        연락처 = cols[8]
        
        value_list.append([거래, 종류, 확인일자, 현장확인, 확인주석, 공급면적, 전용면적, 층, 매물가, 연락처])
        
    cols = ['거래', '종류', '확인일자', '현장확인', '확인주석', '공급면적', '전용면적', '층', '매물가', '연락처']
    df = pd.DataFrame(value_list, columns=cols)
    return df

In [4]:
df = get_naver_realasset('1168010600')
df

,거래,종류,확인일자,현장확인,확인주석,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2017-08-04,5/14,현장확인 대치SK뷰,110.89,84.38,4/14,170000,우리공인중개사02-565-0066
1,매매,아파트,2017-08-03,5/13,"현장확인 선경1,2차",182.22,160.76,7/15,265000,아성공인중개사사무소02-555-9918
2,매매,아파트,2017-08-01,5/13,"현장확인 한보미도맨션1,2차",220.92,191.06,8/14,320000,미도금성공인중개사사무소02-3453-8885
3,매매,아파트,2017-08-01,5/14,"현장확인 선경1,2차",137.45,128.32,8/15,207000,황금공인중개사사무소02-558-4949
4,매매,아파트,2017-08-01,5/13,현장확인 개포우성1차,102.48,84.81,3/15,178000,황금공인중개사사무소02-558-4949
5,매매,아파트,2017-08-01,5/14,현장확인 대치SK뷰,110.89,84.38,16/17,170000,대치SK VIEW공인중개사사무소02-555-0230
6,매매,아파트,2017-07-28,5/14,현장확인 대치SK뷰,122.42,93.40,10/17,187000,금강공인중개사사무소02-565-8088
7,매매,아파트,2017-07-28,5/14,"현장확인 선경1,2차",148.60,127.75,5/15,220000,대성공인중개사사무소02-556-8888
8,매매,아파트,2017-07-25,5/13,현장확인 대치SK뷰,122.42,93.40,6/20,186000,우리공인중개사02-565-0066
9,매매,아파트,2017-07-25,5/13,현장확인 대치SK뷰,145.94,112.36,5/14,240000,삼성플러스공인중개사사무소02-554-5985


# 페이지 처리

요청 url 에 "&page=페이지번호" 인자를 추가하여 페이지를 처리한다.

In [5]:
import re
import numpy as np
import pandas as pd
import requests
from datetime import datetime
from bs4 import BeautifulSoup

def get_naver_realasset(area_code, page=1):
    url = 'http://land.naver.com/article/articleList.nhn?' \
        + 'rletTypeCd=A01&tradeTypeCd=A1&hscpTypeCd=A01%3AA03%3AA04' \
        + '&cortarNo=' + area_code \
        + '&page=' + str(page)

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')

    table = soup.find('table')
    trs = table.tbody.find_all('tr')
    if '등록된 매물이 없습니다' in trs[0].text:
        return pd.DataFrame()
    
    value_list = []

    # 거래, 종류, 확인일자, 확인주석, 매물명, 면적(㎡), 층, 매물가(만원), 연락처
    for tr in trs[::2]:
        tds = tr.find_all('td')
        cols = [' '.join(td.text.strip().split()) for td in tds]

        if '_thumb_image' not in tds[3]['class']: # 현장확인 날짜와 이미지가 없는 행
            cols.insert(3, '')

        # print(cols)
        거래 = cols[0]
        종류 = cols[1]
        확인일자 = datetime.strptime(cols[2], '%y.%m.%d.')
        현장확인 = cols[3]
        확인주석 = cols[4]
        면적 = cols[5]
        공급면적 = re.findall('공급면적(.*?)㎡', 면적)[0].replace(',', '')
        전용면적 = re.findall('전용면적(.*?)㎡', 면적)[0].replace(',', '')
        공급면적 = float(공급면적)
        전용면적 = float(전용면적)
        층 = cols[6]
        if cols[7].find('호가일뿐 실거래가로확인된 금액이 아닙니다') >= 0:
            pass # 단순호가 별도 처리하고자 하면 내용 추가
        매물가 = int(cols[7].split(' ')[0].replace(',', '')) 
        연락처 = cols[8]
        
        value_list.append([거래, 종류, 확인일자, 현장확인, 확인주석, 공급면적, 전용면적, 층, 매물가, 연락처])
        
    cols = ['거래', '종류', '확인일자', '현장확인', '확인주석', '공급면적', '전용면적', '층', '매물가', '연락처']
    df = pd.DataFrame(value_list, columns=cols)
    return df

In [6]:
df = get_naver_realasset('1168010600', 60) # 60 페이지
df

,거래,종류,확인일자,현장확인,확인주석,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2017-07-07,,래미안대치팰리스1단지,125.49,94.49,15/29,190000,뉴서울공인중개사사무소 02-555-2882
1,매매,아파트,2017-07-07,,동부센트레빌,178.76,145.83,28/29,225000,굿모닝공인중개사02-555-7755
2,매매,재건축,2017-07-07,,은마,115.54,84.43,5/14,140000,금마로공인중개사사무소02-563-3600
3,매매,아파트,2017-07-07,,세영팔래스,103.37,84.85,8/13,93000,신현대부동산공인중개사02-555-8090
4,매매,아파트,2017-07-07,,동부센트레빌,152.54,121.74,12/29,208000,동부삼성공인중개사02-557-5688
5,매매,아파트,2017-07-07,,동부센트레빌,202.99,161.47,6/26,258000,뉴동부 공인중개사02-552-6006
6,매매,아파트,2017-07-07,,동부센트레빌,178.76,145.83,19/29,225000,동부아이파크공인중개사02-564-8949
7,매매,아파트,2017-07-07,,대치삼성1차,108.21,84.58,8/11,125000,홈플러스공인02-564-2459
8,매매,아파트,2017-07-07,,동부센트레빌,178.76,145.83,20/29,225000,동부삼성공인중개사02-557-5688
9,매매,아파트,2017-07-07,,"한보미도맨션1,2차",113.31,84.48,7/14,158000,연세공인중개사사무소02-3453-6800


# 페이지묶어 DataFrame 생성하기
페이지별로 생성된 DataFrame 을 묶어(append) 하나의 DataFrame으로 합친다

In [7]:
area_code = '1168010600' # 강남구, 대치동

df = pd.DataFrame()
for i in range(1, 100): # 최대 100 페이지
    df_tmp = get_naver_realasset(area_code, i)
    if len(df_tmp) <= 0:
        break
    df = df.append(df_tmp, ignore_index=True)

df.head()

,거래,종류,확인일자,현장확인,확인주석,공급면적,전용면적,층,매물가,연락처
0,매매,아파트,2017-08-04,5/14,현장확인 대치SK뷰,110.89,84.38,4/14,170000,우리공인중개사02-565-0066
1,매매,아파트,2017-08-03,5/13,"현장확인 선경1,2차",182.22,160.76,7/15,265000,아성공인중개사사무소02-555-9918
2,매매,아파트,2017-08-01,5/13,"현장확인 한보미도맨션1,2차",220.92,191.06,8/14,320000,미도금성공인중개사사무소02-3453-8885
3,매매,아파트,2017-08-01,5/14,"현장확인 선경1,2차",137.45,128.32,8/15,207000,황금공인중개사사무소02-558-4949
4,매매,아파트,2017-08-01,5/13,현장확인 개포우성1차,102.48,84.81,3/15,178000,황금공인중개사사무소02-558-4949


In [8]:
'강남구, 대치동(1168010600)의 물건 총 %d건' % len(df)

'강남구, 대치동(1168010600)의 물건 총 1906건'

In [9]:
df.tail()

,거래,종류,확인일자,현장확인,확인주석,공급면적,전용면적,층,매물가,연락처
1901,매매,아파트,2017-07-17,,대치한신휴플러스,106.05,84.99,고/9,90000,청담수공인중..02-3446-4484
1902,매매,아파트,2017-07-15,,개포우성1차,102.48,84.81,저/15,175000,동부팰리스공..02-564-0002
1903,매매,아파트,2017-07-12,,대치풍림아이원2차,125.14,115.22,저/7,87500,둥지공인중개..02-558-0114
1904,매매,주상복합,2017-07-10,,대치대우아이빌명문가7차,41.88,33.06,8/11,30500,파란공인중개..02-567-8249
1905,매매,아파트,2017-07-10,,래미안대치하이스턴,142.62,110.39,11/13,150000,선우공인중개..02-566-0700


----
2017 FinanceData http://financedata.github.com , http://fb.com/financedata